# The Model Using the food-101 Dataset to Classify Images of Food

We will first preprocess the data

In [1]:
import tensorflow_datasets as tfds

list_datasets = tfds.list_builders()
print('food101' in list_datasets)

c:\Users\Luqman\Pic2Plate\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [2]:
(train_data,test_data),df_info = tfds.load('food101',
                                           split=['train','validation'], # in our case we have training and validation but some dataset have testing too
                                           shuffle_files=True, # we are shuffle our files to some randomness in our data
                                           as_supervised=True, # beacuse our dataset is supervised
                                           with_info=True) # getting metadata about our dataset

In [3]:
# Features of the dataset
df_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

### Reading all the names of the 101 different types of foods we are training on

In [4]:
# Getting the class names

class_names = df_info.features['label'].names
class_names



['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheesecake',
 'cheese_plate',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

### Preprocess the image data

In [5]:
import tensorflow as tf


def preprocess_image_data(images,labels,img_size=(224,224)):
  '''
  This function will reduce image size and change dtype int float32

  '''
  image  = tf.image.resize(images,img_size)

  return tf.cast(image,dtype=tf.float32),labels

In [6]:
# we are going to use map method to map our preprocess_image_data function for preprocessing

train_data = train_data.map(map_func=preprocess_image_data,num_parallel_calls=tf.data.AUTOTUNE)

# making into batches for training data

train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# preprocess and making batches for testing data

test_data  = test_data.map(map_func=preprocess_image_data,num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [7]:
train_data,test_data

(<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>,
 <PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>)

In [8]:
import os
import datetime
# creating the tensoboard callback
def create_tensorboard_callback(dir_name,experiment):
  date_time = datetime.datetime.now().strftime('%Y/%m/%d:%H-%M-%S')
  path = os.path.join(dir_name,experiment,date_time)
  return tf.keras.callbacks.TensorBoard(log_dir=path)

In [9]:
# model check point callback

checkpoint_path = 'model_checkpoints/cp.cpkt'

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         monitor='val_accuracy',
                                                         save_best_only=True,
                                                         save_weights_only=True,
                                                         verbose=0)

In [10]:
# start by importing it 
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16') # this will do the job

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070, compute capability 8.6


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070, compute capability 8.6


# Modeling

Now, we will import the necessary packages and then work on training the model

In [11]:
mixed_precision.global_policy()

<Policy "mixed_float16">

### Here is where we actually write the code for the convolutional neural network

In [12]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow as tf

# we are going t use the transfer learning - feature extraction

input_shape = (224,224,3)

# our base model will be EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet')
base_model.trainable = False

input = layers.Input(shape=input_shape,name='input_layer')

# data_augmentation = tf.keras.Sequential([ # this is for  data augmentation
#     preprocessing.RandomRotation(0.1),
#     preprocessing.RandomZoom(0.2),
#     preprocessing.RandomFlip(mode='horizontal'),
#     preprocessing.Rescaling(1/255.)
# ], name='data_augmentation')

x = base_model(input)
x = layers.GlobalAveragePooling2D(name='globalAvgPooling_layer')(x)

# x = layers.Dense(512, activation='relu')(x)
# x = layers.Dropout(0.2)(x)

x = layers.Dense(101,name='output_layer')(x)

output = layers.Activation(activation='softmax',dtype=tf.float32,name='prediction')(x)

model = tf.keras.Model(input,output,name='food_vision_model')

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

### Here, we train the CNN

In [13]:
feature_extraction_history = model.fit(train_data,
                                       epochs=5,
                                       steps_per_epoch=len(train_data),
                                       validation_data=test_data,
                                       validation_steps=int(.15*len(test_data)),
                                       callbacks=[checkpoint_callback])

Epoch 1/5
2368/2368 [==============================] - 76s 29ms/step - loss: 1.8870 - accuracy: 0.5422 - val_loss: 1.2400 - val_accuracy: 0.6700
Epoch 2/5
2368/2368 [==============================] - 67s 28ms/step - loss: 1.3654 - accuracy: 0.6483 - val_loss: 1.1328 - val_accuracy: 0.6936
Epoch 3/5
2368/2368 [==============================] - 65s 27ms/step - loss: 1.2291 - accuracy: 0.6793 - val_loss: 1.0944 - val_accuracy: 0.7039
Epoch 4/5
2368/2368 [==============================] - 66s 28ms/step - loss: 1.1393 - accuracy: 0.7004 - val_loss: 1.0807 - val_accuracy: 0.7052
Epoch 5/5
2368/2368 [==============================] - 65s 27ms/step - loss: 1.0737 - accuracy: 0.7165 - val_loss: 1.0614 - val_accuracy: 0.7084


## Evalaute the CNN on the test data

In [14]:
fx_results = model.evaluate(test_data)
fx_results

790/790 [==============================] - 18s 23ms/step - loss: 1.0636 - accuracy: 0.7110


[1.0635861158370972, 0.7109702825546265]

In [15]:
# lets make it up

base_model.trainable = True

for layer in base_model.layers[:-20]:
  layer.trainable = False

In [16]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4), # we are reducing the learning rate so pretrained weights does not change too much
              metrics=['accuracy'])

In [17]:
for layer in model.layers:
  print(layer.name,layer.trainable)

input_layer True
efficientnetb0 True
globalAvgPooling_layer True
output_layer True
prediction True


In [18]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=4) # if val loss decreases for 3 epochs in a row, stop training

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [19]:
# creating the callbacks to reduce learning rate of the model
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=4,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [20]:
best_model = model.fit(train_data,
                       epochs=100,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=int(.15 * len(test_data)),
                       callbacks=[early_stopping,reduce_lr,model_checkpoint])

Epoch 1/100
2368/2368 [==============================] - 77s 31ms/step - loss: 1.1437 - accuracy: 0.6979 - val_loss: 0.9633 - val_accuracy: 0.7362
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets
c:\Users\Luqman\Pic2Plate\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
2368/2368 [==============================] - 73s 30ms/step - loss: 0.8852 - accuracy: 0.7628 - val_loss: 0.9050 - val_accuracy: 0.7495
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets
c:\Users\Luqman\Pic2Plate\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 3/100
2368/2368 [==============================] - 72s 30ms/step - loss: 0.7609 - accuracy: 0.7961 - val_loss: 0.8852 - val_accuracy: 0.7550
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


...

Epoch 5/100
2368/2368 [==============================] - 74s 31ms/step - loss: 0.5811 - accuracy: 0.8448 - val_loss: 0.8741 - val_accuracy: 0.7590
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets
c:\Users\Luqman\Pic2Plate\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 6/100
2368/2368 [==============================] - 72s 30ms/step - loss: 0.5080 - accuracy: 0.8651 - val_loss: 0.8625 - val_accuracy: 0.7675
INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints\assets
c:\Users\Luqman\Pic2Plate\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 7/100
2368/2368 [==============================] - 72s 30ms/step - loss: 0.4483 - accuracy: 0.8840 - val_loss: 0.8673 - val_accuracy: 0.7696
Epoch 8/100
2368/2368 [==============================] - 71s 30ms/step - loss: 0.3933 - accuracy: 0.8981 - val_loss: 0.8844 - val_accuracy: 0.7656
Epoch 9/100
2368/2368 [==============================] - 71s 30ms/step - loss: 0.3508 - accuracy: 0.9109 - val_loss: 0.8869 - val_accuracy: 0.7691
Epoch 10/100
2368/2368 [==============================] - 73s 31ms/step - loss: 0.3085 - accuracy: 0.9234 - val_loss: 0.8914 - val_accuracy: 0.7651

Epoch 00010: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.


In [21]:
model.save(r'C:\Users\Luqman\Summer-2023-ML-Project\model')

INFO:tensorflow:Assets written to: C:\Users\Luqman\Summer-2023-ML-Project\model\assets


INFO:tensorflow:Assets written to: C:\Users\Luqman\Summer-2023-ML-Project\model\assets
c:\Users\Luqman\Pic2Plate\.venv\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# GPU Check


In [22]:
import tensorflow as tf


# Check if GPU is available
print("Is a GPU available: ", tf.config.list_physical_devices('GPU'))

# Check which device GPU is using
print("GPU device name: ", tf.test.gpu_device_name())

tf.config.list_physical_devices('GPU')



Is a GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU device name:  /device:GPU:0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]